# **Apigee GenAI Workshop**

<table align="left">
    <td style="text-align: center">
      <a href="https://colab.research.google.com/github/GoogleCloudPlatform/apigee-samples/blob/genai-workshop/genai-workshop.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo\"><br> Open in Colab
      </a>
    </td>
    <td style="text-align: center">
      <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fapigee-samples%2Fgenai-workshop%2Fgenai-workshop.ipynb">
        <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
      </a>
    </td>    
    <td style="text-align: center">
      <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/apigee-samples/genai-workshop/genai-workshop.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
      </a>
    </td>
    <td style="text-align: center">
      <a href="https://github.com/GoogleCloudPlatform/apigee-samples/blob/genai-workshop/genai-workshop.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
      </a>
    </td>
</table>
<br />
<br />
<br />

## Introduction

Welcome to Google's Apigee GenAI Workshop! 

This hands-on workshop will equip you with the knowledge and skills to leverage the power of Generative AI within your API ecosystem. Through practical exercises and real-world examples, you'll learn how to seamlessly integrate Large Language Models (LLMs) with Apigee, Google's leading API management platform. Get ready to unlock new possibilities and explore the exciting world of GenAI and APIs!

You should already have a lab instance up and running with all the necessary artifacts (Apigee, Vertex AI, Vertex DB, etc) provisioned for you to use. 

First, lets install the necessary dependencies to run the labs

## Install dependencies

In [ ]:
!pip install langchain
!pip install langchain-community
!pip install langchain_google_vertexai
!pip install langchain-openai
!pip install google-cloud-aiplatform
!pip install google-cloud-tasks
!pip install openai

## Initialize notebook variables

You can fetch all the variables from your lab instance

* **PROJECT_ID**: The default GCP project to use when making Vertex API calls.
* **REGION**: The default location to use when making API calls.
* **APIGEE_HOSTNAME**:  The hostname of your Apigee instance

In [11]:
# Define project information
PROJECT_ID = ""  # @param {type:"string"}
LOCATION = ""  # @param {type:"string"}
APIGEE_HOSTNAME = "" # @param {type:"string"}

---

## Lab: LLM-Logging with Apigee

Logging both prompts and candidate responses from large language models (LLMs) allows for detailed analysis and improvement of the model's performance over time. By examining past interactions, AI practitioners can identify patterns leading to refinements in the training data or model architecture. Furthermore, by examining the prompts security teams can detect malicious intent, such as attempts to extract sensitive information or generate harmful content.

Additionally, logging the generated candidates provides insights into the LLM's behavior and helps identify any biases or vulnerabilities in the model itself. This information can then be used to improve security measures, fine-tune the model, and mitigate potential risks associated with LLM usage.

![image](https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/llm-logging/images/llm-logging.png?raw=1)


### Benefits of Logging with Apigee and Google Cloud Logging

* **Seamless logging**: Effortlessly capture prompts, candidate responses, and metadata without complex coding.
* **Scalable and secure**: Leverage Google Cloud's infrastructure for reliable and secure log management.

### How does it work?

1. Prompt request is receved by an Apigee Proxy.
2. Apigee extracts prompt and candidate responses.
3. Apigee logs prompt and candidate responses to Cloud Logging.

### Test Sample

Apigee allows you to seamlessly send logs to Cloud Logging using native integration with the [Message Logging](https://cloud.google.com/apigee/docs/api-platform/reference/policies/message-logging-policy#cloudloggingelement) policy. This sample also includes a message chunking solution that allows logging very long messages (ex. 1M tokens supported by Gemini) and connecting them together using a unique message identifier.

With the following cell you'll be able to invoke an LLM and both prompt and candidate resposne will be logged in Cloud Logging.

In [ ]:
from langchain_google_vertexai import VertexAI
API_ENDPOINT = "https://"+APIGEE_HOSTNAME+"/v1/samples/llm-logging"
# Initialize Langchain
model = VertexAI(
      project=PROJECT_ID,
      location=LOCATION,
      api_endpoint=API_ENDPOINT,
      api_transport="rest",
      streaming=True,
      model_name="gemini-1.5-pro")

prompts = ["Provide an explanation about the benefits of using sunscreen. Make sure to make it as long as a novel."]

for prompt in prompts:
  print(model.invoke(prompt))

### Explore and Analyze Logs with Cloud Logging

1. Navigate to the Cloud Logging Explorer by [**clicking here**](https://console.cloud.google.com/logs/query?_ga=2.194228271.307340908.1727018794-898542846.1726863667).

2. Set the query filter. Make sure to replace the `PROJECT_ID` with the Apigee project ID:

  ```
  logName="projects/PROJECT_ID/logs/apigee"
  ```
3. Run the query and explore the logs. See example below:

  ![image](https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/llm-logging/images/logs-explorer.png?raw=1)


---